In [13]:
import sys
sys.path.append("C:\ProgramData\Anaconda3\envs\Lab\Lib\site-packages")
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence
from torchtext import data
from torchtext import datasets
import time
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.enable =True
torch.backends.cudnn.benchmark = True

## General Settings

In [14]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 1e-4
BATCH_SIZE = 128
NUM_EPOCHS = 15
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 1

## Dataset

In [15]:
TEXT = data.Field(tokenize='spacy',
                  include_lengths=True) 
LABEL = data.LabelField(dtype=torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state=random.seed(RANDOM_SEED),
                                          split_ratio=0.8)

print(f'Num Train: {len(train_data)}')
print(f'Num Valid: {len(valid_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 20000
Num Valid: 5000
Num Test: 25000


In [16]:
TEXT.build_vocab(train_data,
                 max_size=VOCABULARY_SIZE,
                 vectors='glove.6B.100d',
                 unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 20002
Number of classes: 2


In [17]:
train_loader, valid_loader, test_loader = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    sort_within_batch=True, 
    device=DEVICE)

In [18]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break

Train
Text matrix size: torch.Size([133, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([61, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([42, 128])
Target vector size: torch.Size([128])


## Model

In [19]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, num_layers=1):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(input_size = embedding_dim,
                          hidden_size = hidden_dim,
                          num_layers = num_layers)
        self.out = nn.Linear(hidden_dim, output_dim)
        self.sig = nn.Sigmoid()
        
    def forward(self, text, text_length):
        text = self.embedding(text)
        text_length = text_length.cpu() # 拿回cpu
        text = pack_padded_sequence(text, text_length) #input： pad_sequence 的结果length： mini-batch 中句子的实际长度
        output, hidden = self.rnn(text)
        hidden = hidden[-1]
        output = self.out(hidden)
        output = self.sig(output)
        return output

In [20]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, output_dim)
        self.sig = nn.Sigmoid()
        
    def forward(self, text, text_length):
        text = self.embedding(text)
        text_length = text_length.cpu() # 拿回cpu
        text = pack_padded_sequence(text, text_length) #input： pad_sequence 的结果length： mini-batch 中句子的实际长度
        output, (hidden,_) = self.lstm(text)
        hidden = hidden[-1]
        output = self.out(hidden)
        output = self.sig(output)
        return output

In [21]:
class GRU(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(GRU, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, output_dim)
        self.sig = nn.Sigmoid()
        
    def forward(self, text, text_length):
        text = self.embedding(text)
        text_length = text_length.cpu() # 拿回cpu
        text = pack_padded_sequence(text, text_length) #input： pad_sequence 的结果length： mini-batch 中句子的实际长度
        output, hidden = self.gru(text)
        hidden = hidden[-1]
        output = self.out(hidden)
        output = self.sig(output)
        return output

## Training

In [22]:
print(torch.cuda.is_available())

True


In [10]:

import torch
print(torch.__version__)

1.11.0+cu113


In [11]:

print(torch.cuda_version) #查看cuda版本

11.3


In [23]:
def binary_acc(preds, label):
    preds = torch.round(preds)
    correct = torch.eq(preds, label).float()
    acc = correct.sum() / correct.shape[0]
    return acc

def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    pbar = tqdm(iterator, unit='batch', ascii=True, bar_format='{percentage:3.0f}%|{bar:20}{r_bar}')
    model.train()
    for batch in pbar:
        optimizer.zero_grad()
        (text, text_lengths), label = batch.text, batch.label
        predictions = model(text, text_lengths).squeeze()
        loss = criterion(predictions, label.float())
        acc = binary_acc(predictions, label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# evaluate

In [24]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            (text, text_lengths), label = batch.text, batch.label
            predictions = model(text, text_lengths).squeeze()
            loss = criterion(predictions, label.float())
            acc = binary_acc(predictions, label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [25]:
def fit(model,optimizer,criterion):
    MAX_EPOCHS = 10
    loss_tr_hist = []
    loss_val_hist = []
    accuracy_tr_hist = []
    accuracy_val_hist = []

    best_valid_loss = float('inf')

    for epoch in range(MAX_EPOCHS):
        t0 = time.time()
        print(epoch+1)
        train_loss, train_acc = train(model, train_loader, optimizer, criterion)
        loss_tr_hist.append(train_loss)
        accuracy_tr_hist.append(train_acc)
        
        valid_loss, valid_acc = evaluate(model, valid_loader, criterion)
        loss_val_hist.append(valid_loss)
        accuracy_val_hist.append(valid_acc)

        
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'model.pt')

        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\tValid Loss: {valid_loss:.3f} | Valid Acc: {valid_acc*100:.2f}%')
        
    return loss_tr_hist, loss_val_hist, accuracy_tr_hist, accuracy_val_hist

In [26]:
INPUT_DIM = len(TEXT.vocab)
torch.manual_seed(RANDOM_SEED)


modelrnn1 = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
modelrnn1 = modelrnn1.to(DEVICE)
optimizerrnn1 = torch.optim.Adam(modelrnn1.parameters(), lr=LEARNING_RATE) #256隐藏，单RNN

modelrnn2 = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, 2)
modelrnn2 = modelrnn2.to(DEVICE)
optimizerrnn2 = torch.optim.Adam(modelrnn2.parameters(), lr=LEARNING_RATE) #256隐藏，2RNN

modelrnn3 = RNN(INPUT_DIM, EMBEDDING_DIM, 512, OUTPUT_DIM)
modelrnn3 = modelrnn3.to(DEVICE)
optimizerrnn3 = torch.optim.Adam(modelrnn3.parameters(), lr=LEARNING_RATE) #512隐藏，单RNN

modelrnn4 = RNN(INPUT_DIM, EMBEDDING_DIM, 1024, OUTPUT_DIM)
modelrnn4 = modelrnn4.to(DEVICE)
optimizerrnn4 = torch.optim.Adam(modelrnn4.parameters(), lr=LEARNING_RATE) #1024隐藏，单RNN

modellstm = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
modellstm = modellstm.to(DEVICE)
optimizerlstm = torch.optim.Adam(modellstm.parameters(), lr=LEARNING_RATE) #256隐藏，LSTM

modelgru = GRU(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
modelgru = modelgru.to(DEVICE)
optimizergru = torch.optim.Adam(modelgru.parameters(), lr=LEARNING_RATE) #256隐藏，GRU


criterion = nn.BCELoss()

In [27]:
loss_tr_hist1, loss_val_hist1, accuracy_tr_hist1, accuracy_val_hist1 = fit(modelrnn1,optimizerrnn1,criterion)

modelrnn1.load_state_dict(torch.load('model.pt'))
test_loss, test_acc = evaluate(modelrnn1, test_loader, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

1


100%|####################| 157/157 [00:05<00:00, 26.17batch/s]


	Train Loss: 0.686 | Train Acc: 54.61%
	Valid Loss: 0.680 | Valid Acc: 56.19%
2


100%|####################| 157/157 [00:04<00:00, 33.29batch/s]


	Train Loss: 0.656 | Train Acc: 61.14%
	Valid Loss: 0.661 | Valid Acc: 60.39%
3


...

	Train Loss: 0.583 | Train Acc: 68.94%
	Valid Loss: 0.610 | Valid Acc: 67.36%
9


100%|####################| 157/157 [00:04<00:00, 34.59batch/s]


	Train Loss: 0.547 | Train Acc: 72.40%
	Valid Loss: 0.594 | Valid Acc: 68.71%
10


100%|####################| 157/157 [00:04<00:00, 34.34batch/s]


	Train Loss: 0.568 | Train Acc: 70.76%
	Valid Loss: 0.586 | Valid Acc: 69.86%
Test Loss: 0.584 | Test Acc: 69.76%


In [28]:
loss_tr_hist2, loss_val_hist2, accuracy_tr_hist2, accuracy_val_hist2 = fit(modelrnn2,optimizerrnn2,criterion)

modelrnn2.load_state_dict(torch.load('model.pt'))
test_loss, test_acc = evaluate(modelrnn2, test_loader, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

1


100%|####################| 157/157 [00:07<00:00, 22.19batch/s]


	Train Loss: 0.684 | Train Acc: 54.91%
	Valid Loss: 0.668 | Valid Acc: 58.65%
2


100%|####################| 157/157 [00:06<00:00, 22.66batch/s]


	Train Loss: 0.639 | Train Acc: 63.44%
	Valid Loss: 0.636 | Valid Acc: 64.43%
3


...

	Train Loss: 0.544 | Train Acc: 72.91%
	Valid Loss: 0.631 | Valid Acc: 68.57%
9


100%|####################| 157/157 [00:06<00:00, 23.19batch/s]


	Train Loss: 0.568 | Train Acc: 70.60%
	Valid Loss: 0.628 | Valid Acc: 64.53%
10


100%|####################| 157/157 [00:06<00:00, 23.14batch/s]


	Train Loss: 0.548 | Train Acc: 72.73%
	Valid Loss: 0.645 | Valid Acc: 62.23%
Test Loss: 0.572 | Test Acc: 71.35%


In [29]:
loss_tr_hist3, loss_val_hist3, accuracy_tr_hist3, accuracy_val_hist3 = fit(modelrnn3,optimizerrnn3,criterion)

modelrnn3.load_state_dict(torch.load('model.pt'))
test_loss, test_acc = evaluate(modelrnn3, test_loader, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

1


100%|####################| 157/157 [00:06<00:00, 24.16batch/s]


	Train Loss: 0.680 | Train Acc: 55.50%
	Valid Loss: 0.663 | Valid Acc: 58.98%
2


100%|####################| 157/157 [00:06<00:00, 23.40batch/s]


	Train Loss: 0.639 | Train Acc: 62.90%
	Valid Loss: 0.657 | Valid Acc: 60.51%
3


...

	Train Loss: 0.579 | Train Acc: 69.85%
	Valid Loss: 0.626 | Valid Acc: 64.63%
9


100%|####################| 157/157 [00:06<00:00, 23.88batch/s]


	Train Loss: 0.615 | Train Acc: 65.53%
	Valid Loss: 0.611 | Valid Acc: 67.97%
10


100%|####################| 157/157 [00:06<00:00, 24.21batch/s]


	Train Loss: 0.588 | Train Acc: 68.83%
	Valid Loss: 0.636 | Valid Acc: 63.95%
Test Loss: 0.592 | Test Acc: 69.03%


In [30]:
loss_tr_hist4, loss_val_hist4, accuracy_tr_hist4, accuracy_val_hist4 = fit(modelrnn4,optimizerrnn4,criterion)

modelrnn4.load_state_dict(torch.load('model.pt'))
test_loss, test_acc = evaluate(modelrnn4, test_loader, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

1


100%|####################| 157/157 [00:14<00:00, 10.72batch/s]


	Train Loss: 0.675 | Train Acc: 57.62%
	Valid Loss: 0.666 | Valid Acc: 59.34%
2


100%|####################| 157/157 [00:14<00:00, 10.66batch/s]


	Train Loss: 0.651 | Train Acc: 61.75%
	Valid Loss: 0.651 | Valid Acc: 62.09%
3


...

	Train Loss: 0.569 | Train Acc: 71.04%
	Valid Loss: 0.617 | Valid Acc: 67.17%
9


100%|####################| 157/157 [00:15<00:00, 10.45batch/s]


	Train Loss: 0.596 | Train Acc: 68.34%
	Valid Loss: 0.740 | Valid Acc: 48.98%
10


100%|####################| 157/157 [00:15<00:00, 10.45batch/s]


	Train Loss: 0.618 | Train Acc: 65.42%
	Valid Loss: 0.630 | Valid Acc: 65.29%
Test Loss: 0.622 | Test Acc: 65.74%


In [31]:
loss_tr_hist5, loss_val_hist5, accuracy_tr_hist5, accuracy_val_hist5 = fit(modellstm,optimizerlstm,criterion)

modellstm.load_state_dict(torch.load('model.pt'))
test_loss, test_acc = evaluate(modellstm, test_loader, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

1


100%|####################| 157/157 [00:07<00:00, 19.69batch/s]


	Train Loss: 0.687 | Train Acc: 54.82%
	Valid Loss: 0.681 | Valid Acc: 57.58%
2


100%|####################| 157/157 [00:08<00:00, 19.43batch/s]


	Train Loss: 0.644 | Train Acc: 62.85%
	Valid Loss: 0.593 | Valid Acc: 68.63%
3


...

	Train Loss: 0.377 | Train Acc: 83.91%
	Valid Loss: 0.441 | Valid Acc: 79.55%
9


100%|####################| 157/157 [00:08<00:00, 19.44batch/s]


	Train Loss: 0.369 | Train Acc: 84.52%
	Valid Loss: 0.382 | Valid Acc: 83.09%
10


100%|####################| 157/157 [00:08<00:00, 19.33batch/s]


	Train Loss: 0.338 | Train Acc: 86.13%
	Valid Loss: 0.371 | Valid Acc: 83.89%
Test Loss: 0.389 | Test Acc: 83.08%


In [32]:
loss_tr_hist6, loss_val_hist6, accuracy_tr_hist6, accuracy_val_hist6 = fit(modelgru,optimizergru,criterion)

modelgru.load_state_dict(torch.load('model.pt'))
test_loss, test_acc = evaluate(modelgru, test_loader, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

1


100%|####################| 157/157 [00:06<00:00, 23.05batch/s]


	Train Loss: 0.686 | Train Acc: 54.27%
	Valid Loss: 0.677 | Valid Acc: 58.69%
2


100%|####################| 157/157 [00:06<00:00, 23.43batch/s]


	Train Loss: 0.649 | Train Acc: 62.04%
	Valid Loss: 0.574 | Valid Acc: 69.77%
3


...

	Train Loss: 0.415 | Train Acc: 81.84%
	Valid Loss: 0.419 | Valid Acc: 82.07%
9


100%|####################| 157/157 [00:06<00:00, 23.04batch/s]


	Train Loss: 0.352 | Train Acc: 85.10%
	Valid Loss: 0.416 | Valid Acc: 82.95%
10


100%|####################| 157/157 [00:06<00:00, 22.86batch/s]


	Train Loss: 0.344 | Train Acc: 85.83%
	Valid Loss: 0.377 | Valid Acc: 84.00%
Test Loss: 0.395 | Test Acc: 83.26%


In [33]:
loss_tr_hist1

[0.6859054728678078,
 0.6560195859070796,
 0.6367367650293241,
 0.620517904971056,
 0.6177883197547523,
 0.6281139672182168,
 0.5995649399271437,
 0.5830636732517533,
 0.5473510533761067,
 0.5678144725644665]

In [34]:
x = [i for i in range(1,11)]

In [35]:
df = pd.DataFrame(columns=['epoch','acc1','acc2','acc3','acc4','acc5','acc6','loss1','loss2','loss3','loss4','loss5','loss6'])
df['epoch'] = x
df['acc1'] = accuracy_tr_hist1 
df['acc2'] = accuracy_tr_hist2
df['acc3'] = accuracy_tr_hist3 
df['acc4'] = accuracy_tr_hist4 
df['acc5'] = accuracy_tr_hist5 
df['acc6'] = accuracy_tr_hist6 
df['loss1'] = loss_tr_hist1
df['loss2'] = loss_tr_hist2
df['loss3'] = loss_tr_hist3
df['loss4'] = loss_tr_hist4
df['loss5'] = loss_tr_hist5
df['loss6'] = loss_tr_hist6

In [36]:
df

,epoch,acc1,acc2,acc3,acc4,acc5,acc6,loss1,loss2,loss3,loss4,loss5,loss6
0,1,0.546079,0.549064,0.555036,0.576234,0.548169,0.542695,0.685905,0.683731,0.680234,0.674781,0.687267,0.686225
1,2,0.611365,0.634355,0.628981,0.617486,0.628533,0.620372,0.656020,0.638795,0.638913,0.651162,0.643656,0.649142
2,3,0.635898,0.594049,0.646895,0.635699,0.725418,0.739003,0.636737,0.666902,0.629717,0.636393,0.551418,0.532194
3,4,0.658290,0.629031,0.671228,0.631568,0.758410,0.777866,0.620518,0.642737,0.608046,0.640361,0.509798,0.481293
4,5,0.658290,0.663117,0.683668,0.637789,0.789560,0.805036,0.617788,0.614306,0.593858,0.636752,0.460616,0.434489
5,6,0.648388,0.683469,0.668541,0.687400,0.802448,0.822900,0.628114,0.587425,0.602701,0.594015,0.435883,0.406191
6,7,0.679688,0.718451,0.717805,0.702578,0.826483,0.824990,0.599565,0.559333,0.558950,0.578497,0.402690,0.404400
7,8,0.689441,0.729100,0.698547,0.710440,0.839072,0.818372,0.583064,0.543823,0.578585,0.568770,0.377000,0.414540
8,9,0.723975,0.706011,0.655255,0.683420,0.845193,0.850965,0.547351,0.568166,0.614644,0.595811,0.368619,0.352315
9,10,0.707604,0.727309,0.688346,0.654210,0.861316,0.858280,0.567814,0.548225,0.587620,0.617573,0.337983,0.343533


In [37]:
df.to_csv('data.csv')

In [ ]:
import matplotlib.pyplot as plt
plt.title('')  # 折线图标题
plt.rcParams['font.sans-serif'] = ['SimHei']  # 显示汉字
plt.xlabel('epoch')  # x轴标题
plt.ylabel('Accurance')  # y轴标题
plt.plot(x, accuracy_tr_hist1, marker='o', markersize=1)  # 绘制折线图，添加数据点，设置点的大小
plt.plot(x, accuracy_tr_hist2, marker='o', markersize=1)
plt.plot(x, accuracy_tr_hist3, marker='o', markersize=1)
plt.plot(x, accuracy_tr_hist4, marker='o', markersize=1)
plt.plot(x, accuracy_tr_hist5, marker='o', markersize=1)
plt.plot(x, accuracy_tr_hist6, marker='o', markersize=1)

for a, b in zip(x, accuracy_tr_hist1):
    plt.text(a, b, b, ha='center', va='bottom', fontsize=10)  # 设置数据标签位置及大小
for a, b in zip(x, accuracy_tr_hist2):
    plt.text(a, b, b, ha='center', va='bottom', fontsize=10)
for a, b in zip(x, accuracy_tr_hist3):
    plt.text(a, b, b, ha='center', va='bottom', fontsize=10)
for a, b in zip(x, accuracy_tr_hist4):
    plt.text(a, b, b, ha='center', va='bottom', fontsize=10)
for a, b in zip(x, accuracy_tr_hist5):
    plt.text(a, b, b, ha='center', va='bottom', fontsize=10)
for a, b in zip(x, accuracy_tr_hist6):
    plt.text(a, b, b, ha='center', va='bottom', fontsize=10)

plt.legend(['256隐藏，单RNN', '256隐藏，2RNN，单RNN', '512隐藏，单RNN', '1024隐藏，单RNN', '256隐藏，LSTM','256隐藏，GRU'])  # 设置折线名称
plt.savefig('acc.pdf')
plt.show()  # 显示折线图


In [ ]:
import matplotlib.pyplot as plt
plt.title('')  # 折线图标题
plt.rcParams['font.sans-serif'] = ['SimHei']  # 显示汉字
plt.xlabel('epoch')  # x轴标题
plt.ylabel('Loss')  # y轴标题
plt.plot(x, loss_tr_hist1, marker='o', markersize=1)  # 绘制折线图，添加数据点，设置点的大小
plt.plot(x, loss_tr_hist2, marker='o', markersize=1)
plt.plot(x, loss_tr_hist3, marker='o', markersize=1)
plt.plot(x, loss_tr_hist4, marker='o', markersize=1)
plt.plot(x, loss_tr_hist5, marker='o', markersize=1)
plt.plot(x, loss_tr_hist6, marker='o', markersize=1)

for a, b in zip(x, loss_tr_hist1):
    plt.text(a, b, b, ha='center', va='bottom', fontsize=10)  # 设置数据标签位置及大小
for a, b in zip(x, loss_tr_hist2):
    plt.text(a, b, b, ha='center', va='bottom', fontsize=10)
for a, b in zip(x, loss_tr_hist3):
    plt.text(a, b, b, ha='center', va='bottom', fontsize=10)
for a, b in zip(x, loss_tr_hist4):
    plt.text(a, b, b, ha='center', va='bottom', fontsize=10)
for a, b in zip(x, loss_tr_hist5):
    plt.text(a, b, b, ha='center', va='bottom', fontsize=10)
for a, b in zip(x, loss_tr_hist6):
    plt.text(a, b, b, ha='center', va='bottom', fontsize=10)

plt.legend(['256隐藏，单RNN', '256隐藏，2RNN，单RNN', '512隐藏，单RNN', '1024隐藏，单RNN', '256隐藏，LSTM','256隐藏，GRU'])  # 设置折线名称
plt.savefig('loss.pdf')
plt.show()  # 显示折线图
